# OOD Detection Pipeline

## 1. Load data to obtain hidden representations.

### Datasets

In [1]:
from constants import JORDAN_DATASET_FILEPATH, OOD_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.ood_dataset import MaestroDataset


print("Loading Jordan dataset - train")
id_train_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train"
)
print(f"Obtained {len(id_train_dataset)} training examples from Jordan dataset - train")

print("Loading Jordan dataset - test")
id_test_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation"
)
print(f"Obtained {len(id_test_dataset)} testing examples from Jordan dataset - test")

print("Loading Maestro dataset - test")
ood_dataset = MaestroDataset(
    data_dir=OOD_DATASET_FILEPATH,
    split="test"
)
print(f"Obtained {len(ood_dataset)} testing examples from Maestro dataset - test")

/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Jordan dataset - train
Loading train split from /scratch/joel/jordan_dataset...
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
Something might be wrong, skipping
So

### Model

In [2]:
from constants import DEVICE, JORDAN_MODEL_NAME
import torch
from transformers import AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained(
    JORDAN_MODEL_NAME,
    dtype=torch.float32,
).to(DEVICE)

### Extract layers

In [6]:
from extract_layers.pooling_functions import pool_mean_std

batch_size = 8
n_layers = 24
pooling_function = pool_mean_std

layers_to_extract = list(range(n_layers + 1))

In [7]:
from constants import SCRATCH_FILEPATH
from torch.utils.data import DataLoader
from utils.data_loading import collate_fn
from extract_layers.extract_layers_main import extract_representations

for dataset_name, dataset in [("id_train_dataset", id_train_dataset), ("id_test_dataset", id_test_dataset), ("ood_dataset", ood_dataset)]:
    print(f"Creating {dataset_name} dataloader")
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    print(f"Extracting representations from {dataset_name} dataloader")
    save_dir = f"{SCRATCH_FILEPATH}/{dataset_name}"
    extract_representations(
        model,
        dataloader,
        pooling_function=pooling_function,
        save_dir=save_dir,
        layers=layers_to_extract,
    )


Creating id_train_dataset dataloader
Extracting representations from id_train_dataset dataloader
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(55028, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Line

Extracting hidden states:   0%|          | 0/508 [00:00<?, ?it/s]

25 torch.Size([8, 766, 1024])


Extracting hidden states:   0%|          | 1/508 [00:00<01:49,  4.61it/s]

25 torch.Size([8, 766, 1024])


...

25 torch.Size([8, 1024, 1024])


Extracting hidden states:  98%|█████████▊| 119/121 [00:42<00:00,  2.78it/s]

25 torch.Size([8, 1024, 1024])


Extracting hidden states: 100%|██████████| 121/121 [00:43<00:00,  2.80it/s]

25 torch.Size([2, 1024, 1024])
Saved layer 0: (962, 2048)
Saved layer 1: (962, 2048)
Saved layer 2: (962, 2048)
Saved layer 3: (962, 2048)
Saved layer 4: (962, 2048)
Saved layer 5: (962, 2048)
Saved layer 6: (962, 2048)
Saved layer 7: (962, 2048)
Saved layer 8: (962, 2048)
Saved layer 9: (962, 2048)
Saved layer 10: (962, 2048)
Saved layer 11: (962, 2048)
Saved layer 12: (962, 2048)
Saved layer 13: (962, 2048)
Saved layer 14: (962, 2048)
Saved layer 15: (962, 2048)
Saved layer 16: (962, 2048)
Saved layer 17: (962, 2048)
Saved layer 18: (962, 2048)
Saved layer 19: (962, 2048)
Saved layer 20: (962, 2048)
Saved layer 21: (962, 2048)
Saved layer 22: (962, 2048)
Saved layer 23: (962, 2048)
Saved layer 24: (962, 2048)


# OOD Detector

## Transformations

Refer to [OOD Detection docs](docs/ood_detection.md) for more information on this.

In [ ]:
from main.transformations import Transformations
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

transformations = Transformations(
    [
        PCA(n_components=10),
        StandardScaler(),
    ]
)

In [ ]:
from main.ood_detector import OODDetector

ood_detector = OODDetector(
    embedding_function=embedding_function,
    scoring_function=scoring_function,
    id_train_data=id_train_data,
)